# Node2Vec Script

First the required packages and the bank.csv file is loaded.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import networkx as nx
from node2vec import Node2Vec

df = pd.read_csv('bank.csv',sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


## Recoding

The data is recoded in the following code 

In [2]:
df.job.unique()

array(['unemployed', 'services', 'management', 'blue-collar',
       'self-employed', 'technician', 'entrepreneur', 'admin.', 'student',
       'housemaid', 'retired', 'unknown'], dtype=object)

In [3]:
df.age = df.age.apply(lambda x: 0 if x < 25 else(1 if x < 35 else(2 if x < 50 else(3 if x < 65 else 4))))

#df.job = df.job.apply(lambda x: 0 if x == 'unemployed' else(1 if x == 'services' \
#                            else(2 if x == 'management' else(3 if x == 'blue-collar' \
#                                else(4 if x == 'self-employed' else(5 if x == 'technician' \
#                                    else(6 if x == 'entrepreneur' else(7 if x == 'admin.' \
#                        else(8 if x =='student' else(9 if x == 'housemaid' \
#                            else(10 if x == 'retired' else 11)))))))))))

#df.job = df.job.apply(lambda x: 0 if x == 'unemployed' or 'unknown' else(1 if x == 'services' \
#                        or x == 'management' or x == 'admin.' else(2 if x == 'blue-collar' or x == 'technician' \
#                                    or x == 'housemaid' else(3 if x == 'self-employed' or x == 'entrepreneur' \
#                                        else(4 if x == 'student' else 5)))))

df.job = df.job.apply(lambda x: 0 if x == 'student' else(2 if x == 'retired' else(3 if x == 'unemployed' or x == 'unknown' else 1)))


df.marital = df.marital.apply(lambda x: 0 if x == 'single' else(1 if x == 'married' else 2))

df.education = df.education.apply(lambda x: 0 if x == 'primary' else(1 if x == 'secondary' else(2 if x == 'tertiary' else 3)))

df.default = df.default.apply(lambda x: 0 if x == 'no' else 1)

df.balance = df.balance.apply(lambda x: 0 if x < 0 else(1 if x < 69. else(2 if x < 444. else(3 if x < 1480 else 4))))

df.housing = df.housing.apply(lambda x: 0 if x == 'no' else 1)

df.loan = df.loan.apply(lambda x: 0 if x == 'no' else 1)

df.contact = df.contact.apply(lambda x: 0 if x == 'telephone' else(1 if x == 'cellular' else 2))

df.pdays = df.pdays.apply(lambda x: 0 if x < 50 else 0)

df.poutcome = df.poutcome.apply(lambda x: 0 if x == 'failure' else(1 if x == 'success' else 2))

df.y = df.y.apply(lambda x: 0 if x == 'no' else 1)

df.drop(columns = ['day','month'],inplace=True)


In [4]:
df

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y
0,1,3,1,0,0,4,0,0,1,79,1,0,0,2,0
1,1,1,1,1,0,4,1,1,1,220,1,0,4,0,0
2,2,1,0,2,0,3,1,0,1,185,1,0,1,0,0
3,1,1,1,2,0,3,1,1,2,199,4,0,0,2,0
4,3,1,1,1,0,1,1,0,2,226,1,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,1,1,1,1,0,0,1,0,1,329,5,0,0,2,0
4517,3,1,1,2,1,0,1,1,2,153,1,0,0,2,0
4518,3,1,1,1,0,2,0,0,1,151,11,0,0,2,0
4519,1,1,1,1,0,3,0,0,1,129,4,0,3,2,0


## MAG

The attributes for the MAG are selected below and the graph is thereafter generated.

In [5]:
X = df[['age','job','marital','education','balance','housing','duration']]
X.duration = X.duration.apply(lambda x: 0 if x < 400 else 1)
X

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,age,job,marital,education,balance,housing,duration
0,1,3,1,0,4,0,0
1,1,1,1,1,4,1,0
2,2,1,0,2,3,1,0
3,1,1,1,2,3,1,0
4,3,1,1,1,1,1,0
...,...,...,...,...,...,...,...
4516,1,1,1,1,0,1,0
4517,3,1,1,2,0,1,0
4518,3,1,1,1,2,0,0
4519,1,1,1,1,3,0,0


In [6]:
# age
age_alpha0 = 0.90
age_beta1 = 0.80
age_beta2 = 0.60
age_beta3 = 0.40
age_beta4 = 0.20

aff_age = np.array([[age_alpha0,age_beta1,age_beta2,age_beta3,age_beta4],
                    [age_beta1,age_alpha0,age_beta1,age_beta2,age_beta3],
                    [age_beta2,age_beta1,age_alpha0,age_beta1,age_beta2],
                    [age_beta3,age_beta2,age_beta1,age_alpha0,age_beta1],
                    [age_beta4,age_beta3,age_beta2,age_beta1,age_alpha0]])


# job

job_alpha0 = 0.90 # self
job_beta_wf = 0.60 # workforce
job_beta_nwf = 0.60 # not worforce

#aff_job = np.array([[job_alpha0, job_beta_wf , job_beta_wf , job_beta_wf , job_beta_nwf, job_beta_nwf],
#                    [job_beta_nwf, job_alpha0, job_beta_wf, job_beta_wf, job_beta_nwf, job_beta_nwf],
#                    [job_beta_nwf, job_beta_wf, job_alpha0, job_beta_wf, job_beta_nwf, job_beta_nwf],
#                    [job_beta_nwf, job_beta_wf, job_beta_wf, job_alpha0, job_beta_nwf, job_beta_nwf],
#                    [job_beta_nwf, job_beta_wf, job_beta_wf, job_beta_wf, job_alpha0, job_beta_nwf],
#                    [job_beta_nwf, job_beta_wf, job_beta_wf, job_beta_wf, job_beta_nwf, job_alpha0]])

# employment
emp_alpha0 = 0.90 # self
emp_beta_wf = 0.75 # beta workforce / employed / unemployed vs student
emp_beta_ret = 0.70 # beta retired vs workforce
emp_beta_sr = 0.20 # gen gap student / retired

aff_job = np.array([[emp_alpha0,emp_beta_wf,emp_beta_sr,emp_beta_wf],
                    [emp_beta_wf,emp_alpha0,emp_beta_ret,emp_beta_wf],
                    [emp_beta_sr,emp_beta_ret,emp_alpha0,emp_beta_ret],
                    [emp_beta_wf,emp_beta_wf,emp_beta_ret,emp_alpha0]])


# marital

mar_alpha0 = 0.85
mar_beta1 = 0.65

aff_mar = np.array([[mar_alpha0, mar_beta1, mar_beta1],
                    [mar_beta1, mar_alpha0, mar_beta1],
                    [mar_beta1, mar_beta1, mar_alpha0]])

# education
edu_alpha0 = 0.85
edu_beta1 = 0.70
edu_beta2 = 0.55
edu_beta3 = 0.40

aff_edu = np.array([[edu_alpha0,edu_beta1,edu_beta2,edu_beta3],
                    [edu_beta1,edu_alpha0,edu_beta1,edu_beta2],
                    [edu_beta2,edu_beta1,edu_alpha0,edu_beta1],
                    [edu_beta3,edu_beta2,edu_beta1,edu_alpha0]])


# balance
inc_alpha0 = 0.85
inc_beta1 = 0.70
inc_beta2 = 0.60
inc_beta3 = 0.50
inc_beta4 = 0.40
inc_beta5 = 0.25

aff_bal = np.array([[inc_alpha0,inc_beta1,inc_beta2,inc_beta3,inc_beta4],
                    [inc_beta1,inc_alpha0,inc_beta1,inc_beta2,inc_beta3],
                    [inc_beta2,inc_beta1,inc_alpha0,inc_beta1,inc_beta2],
                    [inc_beta3,inc_beta2,inc_beta1,inc_alpha0,inc_beta1],
                    [inc_beta4,inc_beta3,inc_beta2,inc_beta2,inc_alpha0]])

# housing
hous_alpha0 = 0.60
hous_beta1 = 0.40

aff_hous = np.array([[hous_alpha0,hous_beta1],
                     [hous_beta1,hous_alpha0]])


# duration
dur_alpha0 = 0.70
dur_alpha1 = 0.95
dur_beta1 = 0.25

aff_dur = np.array([[dur_alpha0,dur_beta1],
                  [dur_beta1,dur_alpha1]])

# outcome

#y_alpha1 = 0.95
#y_beta1 = 0.25

#aff_y = np.array([[y_alpha1,y_beta1],
#                  [y_beta1,y_alpha1]])

aff = {}
aff = {0:aff_age, 1:aff_job ,2:aff_mar ,3:aff_edu ,4:aff_bal ,5:aff_hous,6:aff_dur}

In [7]:
x = X[['age','job','marital','education','balance','housing','duration']]
x = x.to_numpy()
x = x.T

P_u_v = np.empty((len(x.T),len(x.T)))
prob_u_v = np.empty((len(x),1))
u = np.empty((len(x),1))
v = np.empty((len(x),1))

# create probability for connection between u and v
for i in range(len(x.T)):
    u = x[:,i]

    for k in range(len(x.T)):
        v = x[:,k]

        for j in range(len(x)):
            aff_current = aff[j]
            prob_u_v[j] = aff_current[u[j],v[j]]

        P_u_v[i,k] = np.prod(prob_u_v) 

In [8]:
A = np.triu(P_u_v,1)

for i in range(len(P_u_v)):
    for j in range(len(P_u_v)):
        if A[i,j] > np.random.rand():
            A[i,j] = 1
        else:
            A[i,j] = 0



In [9]:
A = A + A.T - np.diag(np.diag(A))
np.sum(A) / 2

737032.0

In [10]:
G = nx.from_numpy_matrix(A)

## Node2Vec

Below the node embeddings are generated using the node2vec algorithm

In [11]:
node2vec = Node2Vec(G, dimensions=6, walk_length=30, num_walks=200, workers=1)

Computing transition probabilities:   0%|          | 0/4521 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 200/200 [1:29:36<00:00, 26.88s/it]


In [12]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [13]:
model.wv.save_word2vec_format('embeddings')

The node embeddings are saved and are loaded again. The node embeddings are saved as the node2vec prodedure can take up a lot of time.

In [ ]:
t = pd.read_csv('embeddings', delimiter = " ", index_col=0)
t.sort_index(inplace=True)
t.columns = ['x1','x2','x3','x4']
t = t.to_numpy()
t.shape

## Downstream Machine Learning

Below the node embeddingare used for downstream machine learning

### ANN

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(t,df.Y, test_size = 0.2, shuffle=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

x_train_tf = tf.convert_to_tensor(x_train)
x_test_tf = tf.convert_to_tensor(x_test)
y_train_tf = tf.convert_to_tensor(y_train)
y_test_tf = tf.convert_to_tensor(y_test)

model = Sequential()

model.add(Dense(12, input_dim=4, activation='relu', name='layer1'))
model.add(Dense(8, activation='relu', name='layer2'))
model.add(Dense(1, activation='sigmoid', name='layer3'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train_tf, y_train_tf, validation_data=(x_test_tf,y_test_tf), epochs=10, batch_size=10)

In [ ]:
plt.plot(history.history['loss'],label = 'Training Loss', color = 'blue')
plt.plot(history.history['val_loss'],label = 'Validation Loss', color = 'red')
plt.xlabel('Number of Epochs')
plt.ylabel('Binary Cross-Entropy Loss')
plt.title('Loss per Epoch')
plt.legend()
plt.show()

In [ ]:
model.layers[2].output

In [ ]:
model.summary()

In [ ]:
np.sum(y_train) / len(y_train)

### Support Vector Machine

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(x_train,y_train)
clf.score(x_train,y_train)

### Logistic Regression

In [ ]:
import statsmodels.api as sm

log_reg = sm.Logit(y_train, x_train).fit()
print(log_reg.summary())


In [ ]:
log_reg = sm.Logit(y_train, x_train).fit()
print(log_reg.summary())